In [2]:
import cv2
from skimage.feature import hog
import numpy as np
from keras.models import model_from_json
from keras.models import load_model
from resizeimage import resizeimage

Using TensorFlow backend.


In [3]:
from skimage.feature import hog
import matplotlib.pyplot as plt

In [4]:
datapath = '/Users/samhithraj/Desktop/emnist_byclass_dataset/'
json_file = open(datapath+'model_num.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights(datapath+"model_num.h5")
print("Loaded model from disk")

loaded_model.save('model_num.hdf5')
loaded_model=load_model('model_num.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [23]:
im = cv2.imread('/Users/samhithraj/Desktop/t_images/sm.jpg')          # read in testing numbers image
imcopy=im.copy()
if im is None:                           # if image was not read successfully
    print("error: image not read from file \n\n")       # print error message to std out
    os.system("pause")   

In [24]:

imgGray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# cv2.imshow("0", imgGray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

imgBlurred = cv2.GaussianBlur(imgGray,(5,5), 0)                    # blur
# cv2.imshow("2", imgBlurred)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [25]:
# ret,th1 = cv2.threshold(imgGray,127,255,cv2.THRESH_BINARY)                                                    # filter image from grayscale to black and white
imgThresh = cv2.adaptiveThreshold(imgBlurred,                           # input image
                                  255,                                  # make pixels that pass the threshold full white
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,       # use gaussian rather than mean, seems to give better results
                                  cv2.THRESH_BINARY_INV,                # invert so foreground will be white, background will be black
                                  11,                                   # size of a pixel neighborhood used to calculate threshold value
                                  2)                                    # constant subtracted from the mean or weighted mean
# cv2.imshow("3",imgThresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [26]:
imgMB=cv2.medianBlur(imgThresh,5)
# cv2.imshow("4",imgMB)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
imgMBcopy = imgMB.copy()        # make a copy of the thresh image, this in necessary b/c findContours modifies the image
npaContours, npaHierarchy = cv2.findContours(imgMBcopy,             # input image, make sure to use a copy since the function will modify this image in the course of finding contours
                                             cv2.RETR_EXTERNAL,         # retrieve the outermost contours only
                                             cv2.CHAIN_APPROX_SIMPLE)   # compress horizontal, vertical, and diagonal segments and leave only their end points

In [27]:
contours=npaContours
contours = sorted(contours, key=cv2.contourArea)
areas = []
for c in contours:
    areas.append(cv2.contourArea(c))
print(areas) 
rects=[]
Area=cv2.contourArea(contours[-1])
for ctr in npaContours:
    if cv2.contourArea(ctr)>(Area/6):
        rects.append(cv2.boundingRect(ctr))
    else:
        continue
print(rects)

[1088.5, 1198.5, 1442.5]
[(132, 11, 57, 69), (6, 11, 76, 68), (227, 5, 66, 82)]


In [28]:
rects1=sorted(rects,key = lambda rects: rects[0])

In [29]:
labels = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
result=''
for rect in rects1:
    # Draw the rectangles
    cv2.rectangle(imcopy, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255,),2)
#     roi = imgMBcopy[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
    pt1=rect[0]+((rect[2]-rect[3])//2)
    pt2=rect[0]+((rect[2]+rect[3])//2)
    roi= imgMBcopy[rect[1]:rect[1]+rect[3], pt1:pt2]

    # Resize the image
    
    bord = cv2.copyMakeBorder(roi,10,10,10,10,cv2.BORDER_CONSTANT,value=[0,0,0])
    thresh = cv2.resize(bord, (28,28),interpolation =cv2.INTER_AREA )
#     cv2.imshow("a",thresh)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    npaROIResized = thresh.reshape(1,28,28, 1)      # flatten image into 1d numpy array
    npaROIResized = npaROIResized.astype('float32')
    npaROIResized/=255
    pred = loaded_model.predict(npaROIResized.T)
    pred_letter=labels[int(np.argmax(pred))]
    result=result+pred_letter
    cv2.putText(imcopy,pred_letter,(rect[0],rect[1]),cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 255, 255),2)
print(result)    

SAM


In [30]:
cv2.imshow("Resulting Image with Rectangular ROIs", imcopy)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [31]:
from gtts import gTTS 
  
# This module is imported so that we can  
# play the converted audio 
import os

In [32]:
# Language in which you want to convert 
language = 'en'
t=result
# Passing the text and language to the engine,  
# here we have marked slow=False. Which tells  
# the module that the converted audio should  
# have a high speed 
myobj = gTTS(text=t, lang=language, slow=True) 
  
# Saving the converted audio in a mp3 file named 
# welcome  
myobj.save("/Users/samhithraj/Desktop/wel.mp3") 
  
# Playing the converted file 
os.system("/Users/samhithraj/Desktop/wel.mp3")

32256